In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

arquivo_tweets = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/coronaVirus.csv')
print('arquivos lidos')

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb225f67-fc40-4445-ba6b-2a33e504313e;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 510ms :: artifacts dl 23ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

arquivos lidos


In [2]:
import pyspark.sql.functions as f

count_palavras_1=arquivo_tweets.withColumn('palavra', f.explode(f.split(f.col('Tweet'), ' ')))\
    .groupBy('palavra')\
    .count()\
    .sort('count', ascending=False)\
    .filter(col('palavra')!='')\


In [3]:
count_palavras_1.show()

+-----------+-----+
|    palavra|count|
+-----------+-----+
|      virus|   92|
|     corona|   69|
|        bem|   33|
|   natureza|   31|
|      criou|   31|
|     Corona|   27|
|      Ainda|   19|
|LulaOficial|   14|
|    chamado|   14|
|       Lula|   13|
|  JanjaLula|   12|
|      Virus|   12|
|    monstro|   10|
|      ainda|    9|
|          q|    8|
|         ta|    7|
|     guerra|    6|
|        tao|    5|
|      frase|    5|
|      disse|    5|
+-----------+-----+
only showing top 20 rows



In [20]:
#Salvando S3 estado_IDH
count_palavras_1.write.mode('overwrite').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-final/count_palavras_tweets.csv/')

22/06/06 01:51:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/06 01:51:18 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [8]:
#Salvando Azure - casos e evolução
print("inserindo azure")
count_palavras_1.write \
.format('com.microsoft.sqlserver.jdbc.spark') \
.mode('append') \
.option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
.option('url', 'jdbc:sqlserver://servidor-grupo04.database.windows.net;databaseName=grupo04;') \
.option('dbtable', 'table_count_tweets') \
.option('user', 'urubu100') \
.option('password', 'Urubu1@@') \
.save()
print("Tabela por caso salva")

inserindo azure


Tabela por caso salva
